In [9]:
prompt = input("Hey, how can I help you today?")

print(prompt)

create a simple navbar with home, about and contact links in html, css


In [ ]:
%pip install groq
%pip install langchain-groq

In [ ]:
from groq import Groq
from langchain_groq import ChatGroq

In [ ]:
%pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Simple code snippet to make api calls to use the LLM
client = Groq()

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

In [42]:
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.0,
    max_retries=2,
    verbose=True
)
messages = [
    ("system", "You are a senior engineer working for a professional software company. Your task is to convert ideas into actionable steps. You will be prompted with a general ideas and you need to respond back with how to make it with proper steps instructions."),
    ("human", "An app to track my daily expenses."),
]

In [ ]:
print(llm.invoke(messages))

# To stream
# for chunk in llm.stream(messages):
#     print(chunk.content, end="")

content="Sure, I can help you convert the idea of an app to track daily expenses into actionable steps. Here's a high-level overview of the process:\n\n1. **Define the app's requirements and features**: Start by defining the app's requirements and features. What information do you want to track? Do you want to categorize expenses? Do you want to set budgets? Do you want to generate reports? Make a list of all the features you want to include in the app.\n2. **Choose a platform and technology stack**: Decide which platform you want to build the app for (e.g., iOS, Android, or both). Choose a technology stack that is appropriate for the platform and the app's requirements. For example, you could use Swift and Xcode for an iOS app, or Java and Android Studio for an Android app.\n3. **Design the user interface**: Create wireframes or mockups of the app's user interface. Decide where to place buttons, text fields, and other UI elements. Make sure the app is easy to use and navigate.\n4. **I

In [37]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
import json

In [38]:
# Define the expected JSON structure
parser = JsonOutputParser(pydantic_object={
    "type": "object",
    "properties": {
        "steps": {
            "type": "array",
            "items": {
                "step": {"type": "number"},
                "command": {"type": "string"},
                "description": {
                    "type": "string",
                }
            }
        }
    }
})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """Create a step by step instruction to follow with terminal commands to run on windows to setup a project based on user needs. Understand the user needs if its a frontend project use vite react, if its backend use express
     These commands will be actually be implemented by an autonomous agent so make sure each and every minute details is included and you dont make any mistake:
        {{
        "steps": [{{
            "step": "1",
            "command": "mkdir frontend",
            "description": "Initializing directory"
        }},...]
        }}
     Do not include the actual coding implementation.
     reply with just json and nothing else.
     """),
    ("user", "{input}")
])

In [ ]:
chain = prompt | llm | parser

result = chain.invoke({"input": "I want to make a to do list app named JustDoIt"})
print(json.dumps(result, indent=2))

{
  "steps": [
    {
      "step": "1",
      "command": "mkdir JustDoIt",
      "description": "Creating project directory"
    },
    {
      "step": "2",
      "command": "cd JustDoIt",
      "description": "Navigating into project directory"
    },
    {
      "step": "3",
      "command": "mkdir frontend",
      "description": "Creating frontend directory"
    },
    {
      "step": "4",
      "command": "cd frontend",
      "description": "Navigating into frontend directory"
    },
    {
      "step": "5",
      "command": "npm init -y",
      "description": "Initializing npm"
    },
    {
      "step": "6",
      "command": "npm install vite react react-dom",
      "description": "Installing vite and react dependencies"
    },
    {
      "step": "7",
      "command": "npx vite init",
      "description": "Initializing vite"
    },
    {
      "step": "8",
      "command": "mkdir backend",
      "description": "Creating backend directory"
    },
    {
      "step": "9",
      "c

In [48]:
messages = [
    ("system", "You are a senior engineer who is an expert at finding mistakes. Your task is to find mistakes from a set of actions and give feedback. If all steps feels correct reply back with approved else give feedback."),
    ("human", "here are my step by step idea to make a to do list app named JustDoIt" + json.dumps(result)),
]

llm.invoke(messages)

AIMessage(content='After reviewing your steps, I found a mistake.\n\nIn step 5, you initialized npm in the frontend directory, but you should initialize it in the project root directory (JustDoIt). \n\nHere\'s the corrected step:\n\n{"step": "5", "command": "cd ..", "description": "Navigating back to project root directory"}\n{"step": "6", "command": "npm init -y", "description": "Initializing npm"}\n\nAlso, in step 13, you are initializing the server file, but you should be in the backend directory before running this command. \n\nHere\'s the corrected step:\n\n{"step": "12", "command": "cd backend", "description": "Navigating into backend directory"}\n{"step": "13", "command": "echo \'const express = require(\\"express\\");\\nconst app = express();\\napp.listen(3001, () => {\\n  console.log(\\"Server started on port 3001\\");\\n});\' > server.js", "description": "Initializing server file"}\n\nHere is the corrected set of steps:\n\n{"steps": [{"step": "1", "command": "mkdir JustDoIt",